In [11]:
from importlib import reload
import platform, os, sys, datetime, re
import multiprocessing
from os.path import join
from glob import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# sys.path.append(join(root_dir,'cvtracer'))
import cvt
from cvt.TrAQ.Trial import Trial
from cvt.TrAQ.Tank import Tank
from cvt.TrAQ.CVTracer import CVTracer, create_named_window, wait_on_named_window
from cvt.utils import *

default_settings = dict(
    t_start        = 0,     # Time at which to start tracking, in seconds.
    t_end          = -1,    # Time at which to end tracking, in seconds.
    
    # Background subtraction (for naive background subtraction only).
    bkg_frame_skip = 100,   # Using every frame of the video to compute the background takes a while.
                            # Instead we only use one frame in bkg_frame_skip.
    bkg_sub_amp    = 4,     # Contrast amplification factor applied after background subtraction.
    
    # Contour detection.
    n_pixel_blur   =  7,    # square-root of n-pixels for threshold blurring
    block_size     = 15,    # contour block size
    thresh_offset  = 15,    # threshold offset for contour-finding
    min_area       = 25,    # minimum area for threhold detection
    max_area       = 60,    # maximum area for threhold detection
    RGB            = False, # track in color, false does greyscale
    online_viewer  = False, # Toggle live preview of tracking.

    # What information to draw on the tracking output video.
    video_output_options = dict(tank=True, repeat_contours=False, all_contours=True, 
                                contour_color=(0,200,255), contour_thickness=1, 
                                points=False, directors=True, timestamp=True)
    )

settings_list = list(default_settings.keys()) + \
                ['input_file', 'tracking_dir', 'output_dir', 'trial_file',  
                 'new_input_file', 'bkg_file', 'tank_file', 'settings_file', 
                 'tank_radius', 'ext', 'filename', 'pop', 'age', 'group', 'Nfish', 
                 'Nframes', 'fps', 'fourcc', 'width', 'height']

def create_settings_(input_file, tracking_dir, settings):
    
    globals().update(settings)
    tank_diameter_vs_age = { 7:9.6, 14:10.4, 21:12.8, 28:17.7, 42:33.8 }
    
    ''' Extract trial info from the filename and the video itself. '''

    filename,ext = os.path.splitext(os.path.basename(input_file))
    pop,_,age,group,Nfish = filename.split('_')[:5]
    Nfish    = int(re.findall('\d+',Nfish)[0])
    age      = int(age[:-3])
    tank_radius = tank_diameter_vs_age[age]/2

    cap      = cv2.VideoCapture(input_file)
    cap.read()
    Nframes  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps      = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc   = int(cap.get(cv2.CAP_PROP_FOURCC))
    width    = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height   = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    
    ''' Define and create necessary folders/files/links. '''
    
    output_dir = join(tracking_dir,filename)

    new_input_file = input_file
    if not 'windows' in platform.system().lower():
        new_input_file = join(output_dir,'raw'+ext)
    
    settings_file = join(output_dir,'tracking_settings.txt')
    trial_file    = join(output_dir,'trial.pik')
    tank_file     = join(output_dir,'tank.pik')
    bkg_file      = join(output_dir,f'background-{bkg_frame_skip}.npz')
    
    for k,v in locals().items():
        if k not in ['cap','settings','tank_diameter_vs_age','_']:
            settings[k] = v
        
    return settings


def create_directories(settings):
    for k in 'tracking_dir','output_dir','input_file','new_input_file':
        globals()[k] = settings[k]
    if not os.path.exists(tracking_dir):
        os.mkdir(tracking_dir)
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    if not 'windows' in platform.system().lower():
        if not os.path.exists(new_input_file):
            os.symlink(os.path.relpath(input_file,output_dir),new_input_file)
    return


def save_settings(settings):
    with open(settings['settings_file'],'w') as fh:
        for k,v in settings.items():
            print(f'{k} = {v}',file=fh)
    return

# Locate input/output and allocate CPU's

`tracking_dir` sets the top-level output directory. The output of tracking each video will go in `tracking_dir`, in a subdirectory named after the input video file.  

`n_threads` controls the number of tracking tasks to execute simulataneously. `n_threads = None` defaults to the number of CPU's on the machine running the notebook.

`input_files` set the list of video files to perform tracking on.

In [13]:
# tracking_dir = '../tracking/full' # use this one to track full videos
# tracking_dir = '../tracking/partial' # use this one to track small excerpts
tracking_dir = './test/output' # use this one to track small excerpts

def create_settings(input_file,tracking_dir=tracking_dir,settings=default_settings):
    return create_settings_(input_file,tracking_dir,settings)

n_threads = None

# input_files = sorted(glob('../raw_videos/*.avi'))
# input_files = sorted(glob('../raw_videos/*_*_7dpf_*.avi'))

# input_files = []
# for f in sorted(glob('../raw_videos/*.avi')):
#     print(f)
#     settings = create_settings(f)
#     if not os.path.exists(settings['trial_file']):
#         input_files.append(f)
# display(input_files)

input_files = glob('./test/input/*.avi')
display(input_files)

input_files = [ input_files[0] ]

['./test/input/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000_60-70.avi',
 './test/input/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000_60-120.avi',
 './test/input/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000_60-120.avi',
 './test/input/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000_60-70.avi']

# Locate the tanks

In [14]:
for input_file in input_files:
    print(input_file)
    settings = create_settings(input_file)
    create_directories(settings)
    globals().update(settings)
    tank = Tank(r_cm=tank_radius)
    tank.load_or_locate_and_save(tank_file,input_file)

./test/input/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000_60-70.avi

        Tank object loaded from ./test/output/Pa_Fri_7dpf_GroupA_n5_2020-06-05-083453-0000_60-70/tank.pik 


# Compute the background

### Using a naive average

Compute the background by averaging frames over the entire video. Save it as `background.npy` in the output directory. Use the pre-existing file if there is one.

In [5]:
def subtract_background(frame,bkg,bkg_sub_amp):
    return 255-np.minimum(255,bkg_sub_amp*np.absolute(frame-bkg)).astype(np.uint8)    


def compute_background(settings):
    
    globals().update(settings)
    
    if os.path.exists(bkg_file):
        bkg  = np.load(bkg_file)['bkg']
    else:
        t0    = datetime.datetime.now()
        cap   = cv2.VideoCapture(new_input_file)
        _,frame = cap.read()
        bkg   = np.zeros(frame.shape,dtype=float)
        count = 0
        # If bkg_frame_skip is small (<10) it may be faster to use
        # cap.grab instead of cap.set.
        for n in range(0,Nframes,bkg_frame_skip):
            cap.set(cv2.CAP_PROP_POS_FRAMES,n)
            ret,frame = cap.read()
            bkg      += frame
            count    += 1
        bkg   = bkg / count
        np.savez_compressed(bkg_file,bkg=bkg)
        print(input_file)
        print('Using every {bkg_frame_skip}th frame.')
        print(datetime.datetime.now()-t0)
        sys.stdout.flush()
    
    # Show the background.
    cv2.imwrite(join(output_dir,'background.png'),bkg)
    
    return

#---------------------------------------------------

# Single thread version.
for f in input_files:
    settings = create_settings(f)
    compute_background(settings)

#----

# # Multithread version.
# with multiprocessing.Pool(n_threads) as pool:
#     pool.map(compute_background,[create_settings(f) for f in input_files])

./test/input/Pa_Fri_7dpf_GroupA_n2b_2020-06-05-103456-0000_60-120.avi
Using every {bkg_frame_skip}th frame.
0:00:03.045343


### Using various methods from openCV

In [6]:
globals().update(settings)

from cv2 import createBackgroundSubtractorKNN, createBackgroundSubtractorMOG2
from cv2.bgsegm import *

In [ ]:
for bg in bg_types:
#     help(createBackgroundSubtractorCNT)
    help(globals()['createBackgroundSubtractor'+bg])

In [10]:
Ntraining = 50

bg_types = ['MOG2','KNN','CNT','GMG','GSOC','LSBP','MOG']
# bg_types = ['MOG2','GSOC','CNT']

# bg_sub = cv2.createBackgroundSubtractorMOG2(history=Ntraining) #, varThreshold=25, detectShadows=False)
# bg_sub = createBackgroundSubtractorKNN() #history=10*Ntraining)
# bg_sub = createBackgroundSubtractorCNT()
# bg_sub = createBackgroundSubtractorGMG()
# bg_sub = createBackgroundSubtractorGSOC()
# bg_sub = createBackgroundSubtractorLSBP()
# bg_sub = createBackgroundSubtractorMOG()

bgs_dir = join(output_dir,'backgrounds')
if not os.path.exists(bgs_dir):
    os.mkdir(bgs_dir)

for bg in bg_types:
    
    print(bg)

    cap = cv2.VideoCapture(new_input_file)
    Nframes = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    training_frames = np.linspace(0, Nframes-1, Ntraining, dtype=int)
    
    try:
        opt = dict(history=Ntraining) if bg in ['MOG2','KNN','MOG'] else {}
        bg_sub = globals()['createBackgroundSubtractor'+bg](**opt)

        for i in training_frames:
            cap.set(cv2.CAP_PROP_POS_FRAMES, i)
            ret,frame = cap.read()
    #         print(ret)
            if ret:
                bg_sub.apply(frame,learningRate=1/Ntraining)
        #         cv2.imwrite(f'test/MOG/{i}.png',bg_sub.getBackgroundImage())

        cap.release()
#         cv2.imwrite(f'test/MOG/{bg}-{Ntraining}.png',bg_sub.getBackgroundImage())
        cv2.imwrite(join(bgs_dir,f'{bg}-{Ntraining}.png'),bg_sub.getBackgroundImage())
    except:
        cap.release()
        %tb

MOG2
KNN
CNT
GMG


error: OpenCV(4.2.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:715: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


GSOC


KeyboardInterrupt: 

LSBP


KeyboardInterrupt: 

MOG


KeyboardInterrupt: 